In [ ]:
# ===== CELL 1: SETUP DAN IMPORT =====
!pip install sqlalchemy psycopg2-binary nest_asyncio asyncpg tenacity scikit-learn matplotlib seaborn pandas numpy tabulate

import nest_asyncio
nest_asyncio.apply()

import sys
import os

# Path ke kode sistem rekomendasi
SISTEM_RECOMMENDER_PATH = '../pariwisata-recommender/'
sys.path.append(SISTEM_RECOMMENDER_PATH)

# Import dari kode sistem
try:
    from app.models.rating import Rating
    from app.models.destinations import Destination
    from app.models.category import Category
    from app.models.user import User
    from app.services.collaborative_recommender import CollaborativeRecommender
    from app.services.content_based_recommender import ContentBasedRecommender
    from app.services.hybrid_recommender import HybridRecommender
    from app.services.mab_optimizer import MABOptimizer
    from app.services.real_time_data import RealTimeContextService
    
    print("✅ Berhasil mengimpor kode sistem!")
    KODE_SISTEM_TERSEDIA = True
    
except ImportError as e:
    print(f"⚠️ Tidak bisa mengimpor kode sistem: {e}")
    KODE_SISTEM_TERSEDIA = False

# Import library lainnya
import asyncio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sqlalchemy.ext.asyncio import create_async_engine, AsyncSession
from sqlalchemy.orm import sessionmaker
from sqlalchemy import select, text

In [ ]:
# ===== CELL 2: LOAD DATA =====
async def load_data_ratings():
    """Load data ratings dari database"""
    try:
        async with get_db() as db:
            result = await db.execute(select(Rating))
            ratings = result.scalars().all()
            
            data = []
            for rating in ratings:
                data.append({
                    'user_id': rating.user_id,
                    'destination_id': rating.destination_id, 
                    'rating': float(rating.rating),
                    'created_at': rating.created_at
                })
            
            df = pd.DataFrame(data)
            print(f"✅ Loaded {len(df)} ratings")
            return df
            
    except Exception as e:
        print(f"❌ Gagal load ratings: {e}")
        raise

async def load_data_destinations():
    """Load data destinasi dari database"""
    try:
        async with get_db() as db:
            from sqlalchemy.orm import selectinload
            
            result = await db.execute(
                select(Destination).options(selectinload(Destination.categories))
            )
            destinations = result.scalars().all()
            
            features = {}
            for dest in destinations:
                categories = [cat.name for cat in dest.categories]
                features[dest.id] = {
                    'name': dest.name,
                    'description': dest.description or '',
                    'location': dest.address or '',
                    'categories': categories,
                    'category': categories[0] if categories else 'Umum'
                }
            
            print(f"✅ Loaded {len(features)} destinations dengan categories")
            return features
            
    except Exception as e:
        print(f"❌ Gagal load destinations: {e}")
        raise

# Load data
ratings_df = await load_data_ratings()
destination_features = await load_data_destinations()

In [ ]:
# ===== CELL 2.5: PREPROCESSING DATA UNTUK COLLABORATIVE FILTERING =====
print("🔧 PREPROCESSING DATA UNTUK COLLABORATIVE FILTERING...")

# 1. Identifikasi dan handle duplikat
print("1. Memeriksa duplikat data...")
duplicate_mask = ratings_df.duplicated(subset=['user_id', 'destination_id'], keep=False)
duplicate_count = duplicate_mask.sum()

print(f"   Ditemukan {duplicate_count} entri duplikat")

if duplicate_count > 0:
    print("   Contoh data duplikat:")
    duplicates = ratings_df[duplicate_mask].sort_values(['user_id', 'destination_id'])
    print(duplicates.head(10))
    
    # Group by dan ambil rata-rata rating untuk duplikat
    print("   Menggabungkan duplikat dengan rata-rata rating...")
    ratings_df_clean = ratings_df.groupby(['user_id', 'destination_id'])['rating'].mean().reset_index()
else:
    ratings_df_clean = ratings_df.copy()

print(f"   Data setelah cleaning: {len(ratings_df_clean)} ratings")

# 2. Filter user yang memiliki minimal 2 rating untuk evaluasi yang valid
print("2. Memfilter user dengan minimal 2 rating...")
user_rating_counts = ratings_df_clean['user_id'].value_counts()
users_with_min_ratings = user_rating_counts[user_rating_counts >= 2].index
ratings_df_filtered = ratings_df_clean[ratings_df_clean['user_id'].isin(users_with_min_ratings)]

print(f"   User dengan minimal 2 rating: {len(users_with_min_ratings)}")
print(f"   Ratings setelah filter user: {len(ratings_df_filtered)}")

# 3. Pastikan tidak ada duplikat lagi
final_duplicates = ratings_df_filtered.duplicated(subset=['user_id', 'destination_id']).sum()
print(f"3. Duplikat setelah processing: {final_duplicates}")

if final_duplicates > 0:
    print("   ⚠️ Masih ada duplikat, melakukan cleanup final...")
    ratings_df_filtered = ratings_df_filtered.drop_duplicates(subset=['user_id', 'destination_id'])

# 4. Simpan data yang sudah dibersihkan untuk digunakan model
ratings_df_processed = ratings_df_filtered.copy()

print("✅ PREPROCESSING SELESAI!")
print(f"📊 Data final: {len(ratings_df_processed)} ratings")
print(f"📊 Unique users: {ratings_df_processed['user_id'].nunique()}")
print(f"📊 Unique destinations: {ratings_df_processed['destination_id'].nunique()}")

# Buat DataFrame untuk training collaborative
collab_training_df = ratings_df_processed[['user_id', 'destination_id', 'rating']].copy()

In [ ]:
# ===== CELL 3: INISIALISASI MODEL =====
print("🚀 Inisialisasi model...")

# Gunakan data yang SUDAH DIPROSES dari cell 2.5
ratings_df_for_split = ratings_df_processed.copy()

print(f"📊 Data untuk training: {len(ratings_df_for_split)} ratings")
print(f"📊 Unique users: {ratings_df_for_split['user_id'].nunique()}")
print(f"📊 Unique destinations: {ratings_df_for_split['destination_id'].nunique()}")

# Split data
train_df, test_df = train_test_split(
    ratings_df_for_split, 
    test_size=0.2,
    random_state=42
)

print(f"📊 Training set: {len(train_df)} ratings")
print(f"📊 Test set: {len(test_df)} ratings")

# Inisialisasi model
cf_model = CollaborativeRecommender()
cb_model = ContentBasedRecommender()
hybrid_model = HybridRecommender()
mab_optimizer = MABOptimizer(
    n_arms=11,
    exploration_param=2.0,
    persistence_file="data/mab_state.json"
)
context_service = RealTimeContextService()

print("✅ Model berhasil diinisialisasi!")

# Train model
print("🏋️ Melatih model...")

try:
    async with get_db() as db:
        # Train Collaborative Filtering 
        print("  - Training Collaborative Filtering dengan NMF...")
        try:
            result_cf = await cf_model.train(db)
            print(f"  ✅ Collaborative: {result_cf['users_count']} users, {result_cf['items_count']} items")
            print(f"     Matrix: {result_cf['matrix_shape']}, NMF components: {result_cf['nmf_components']}")
        except Exception as e:
            print(f"  ❌ Collaborative failed: {e}")
        
        # Train Content-Based
        print("  - Training Content-Based...")
        try:
            result_cb = await cb_model.train(db)
            print("  ✅ Content-Based trained")
        except Exception as e:
            print(f"  ❌ Content-Based failed: {e}")
        
        # Train Hybrid  
        print("  - Training Hybrid...")
        try:
            result_hybrid = await hybrid_model.train(db)
            print("  ✅ Hybrid trained")
        except Exception as e:
            print(f"  ❌ Hybrid failed: {e}")

    print("🎯 Training completed!")

except Exception as e:
    print(f"⚠️ Training error: {e}")

In [ ]:
# ===== CELL DEBUG: CEK DATA DI DATABASE =====
async def check_database_duplicates():
    """Cek duplikat langsung di database"""
    async with get_db() as db:
        # Query untuk cek duplikat
        result = await db.execute("""
            SELECT user_id, destination_id, COUNT(*) as count 
            FROM ratings 
            GROUP BY user_id, destination_id 
            HAVING COUNT(*) > 1
            LIMIT 10
        """)
        duplicates = result.fetchall()
        
        print("🔍 DUPLIKAT DI DATABASE:")
        for dup in duplicates:
            user_id, dest_id, count = dup
            print(f"   User {user_id}, Destination {dest_id}: {count} entries")
            
        return len(duplicates)

duplicate_count = await check_database_duplicates()
print(f"Total duplikat di database: {duplicate_count}")

In [ ]:
# ===== CELL 4: EVALUATION FRAMEWORK =====
class ComprehensiveEvaluator:
    def __init__(self, train_df, test_df, item_features):
        self.train_df = train_df
        self.test_df = test_df
        self.item_features = item_features
        self.total_items = len(item_features)
        
        # Calculate item popularity from training data
        self.item_popularity = train_df['destination_id'].value_counts().to_dict()
        self.total_ratings = len(train_df)
        
        # Available categories for diversity calculation
        self.available_categories = set()
        for features in item_features.values():
            if 'category' in features:
                self.available_categories.add(features['category'])
    
    def precision_at_k(self, recs_ids, ground_truth_ids, k=10):
        if not recs_ids or k == 0:
            return 0.0
        recs_ids = recs_ids[:k]
        return len(set(recs_ids) & set(ground_truth_ids)) / min(k, len(recs_ids))
    
    def recall_at_k(self, recs_ids, ground_truth_ids, k=10):
        if not recs_ids or not ground_truth_ids:
            return 0.0
        recs_ids = recs_ids[:k]
        return len(set(recs_ids) & set(ground_truth_ids)) / len(ground_truth_ids)
    
    def ndcg_at_k(self, recs_ids, ground_truth_ids, k=10):
        if not recs_ids or not ground_truth_ids:
            return 0.0
            
        recs_ids = recs_ids[:k]
        
        # Calculate DCG
        dcg = 0.0
        for i, did in enumerate(recs_ids):
            if did in ground_truth_ids:
                dcg += 1 / np.log2(i + 2)
                
        # Calculate IDCG
        ideal_hits = min(len(ground_truth_ids), k)
        idcg = sum(1 / np.log2(i + 2) for i in range(ideal_hits))
        
        return dcg / idcg if idcg > 0 else 0.0
    
    def intra_list_diversity(self, recs_ids, k=10):
        if len(recs_ids) < 2:
            return 0.0
            
        recs_ids = recs_ids[:k]
        
        # Count categories
        category_counts = {}
        for item_id in recs_ids:
            if item_id in self.item_features:
                category = self.item_features[item_id].get('category', 'Unknown')
                category_counts[category] = category_counts.get(category, 0) + 1
        
        # Calculate diversity as inverse of category concentration
        total = sum(category_counts.values())
        if total == 0:
            return 0.0
            
        category_proportions = [count / total for count in category_counts.values()]
        
        # Gini-based diversity (1 - Gini coefficient)
        if len(category_proportions) > 1:
            category_proportions.sort()
            n = len(category_proportions)
            index = np.arange(1, n + 1)
            gini = (np.sum((2 * index - n - 1) * np.array(category_proportions))) / (n * np.sum(category_proportions))
            return 1 - gini
        else:
            return 0.0
    
    def coverage_at_k(self, all_recommendations, k=10):
        all_recommended_items = set()
        
        for recs in all_recommendations:
            all_recommended_items.update(recs[:k])
            
        return len(all_recommended_items) / self.total_items
    
    def novelty(self, recs_ids, k=10):
        if not recs_ids:
            return 0.0
            
        recs_ids = recs_ids[:k]
        novelty_score = 0.0
        valid_items = 0
        
        for item_id in recs_ids:
            popularity = self.item_popularity.get(item_id, 0)
            popularity_norm = popularity / self.total_ratings if self.total_ratings > 0 else 0
            if popularity_norm > 0:
                novelty_score += -math.log2(popularity_norm)
                valid_items += 1
        
        return novelty_score / valid_items if valid_items > 0 else 0.0
    
    def gini_coefficient(self, all_recommendations):
        item_rec_counts = {}
        
        for recs in all_recommendations:
            for item_id in recs:
                item_rec_counts[item_id] = item_rec_counts.get(item_id, 0) + 1
        
        if not item_rec_counts:
            return 0.0
            
        counts = np.array(sorted(item_rec_counts.values()))
        n = len(counts)
        index = np.arange(1, n + 1)
        gini = (np.sum((2 * index - n - 1) * counts)) / (n * np.sum(counts))
        return gini
    
    def calculate_diversity_reward(self, recs_ids, ground_truth_ids, k=10):
        metrics_weights = {
            'precision': 0.1,
            'diversity': 0.7,
            'novelty': 0.2
        }
    
        precision = self.precision_at_k(recs_ids, ground_truth_ids, k)
        diversity = self.intra_list_diversity(recs_ids, k)
        novelty = self.novelty(recs_ids, k)

        # Composite reward dengan emphasis on diversity
        reward = (metrics_weights['precision'] * precision + 
                  metrics_weights['diversity'] * diversity + 
                  metrics_weights['novelty'] * novelty)
    
        return max(reward, 0.01)

print("✅ Evaluation framework defined!")

In [ ]:
# ===== CELL 5: DATA LOADING & PREPROCESSING =====
async def load_ratings_data():
    """Load data dari database atau gunakan synthetic data"""
    try:
        # Coba load dari database
        async with get_db() as db:
            from app.models.rating import Rating
            result = await db.execute(select(Rating))
            rows = result.scalars().all()
        
        data = [{'user_id': r.user_id, 'destination_id': r.destination_id, 'rating': float(r.rating)} for r in rows]
        ratings_df = pd.DataFrame(data)
        print(f"✅ Loaded {len(ratings_df)} ratings from database")
        
    except Exception as e:
        print(f"⚠️ Cannot load from database: {e}. Using synthetic data...")
        
        # Generate synthetic data
        n_users = 100
        n_destinations = 50
        n_ratings = 1000
        
        user_ids = list(range(1, n_users + 1))
        destination_ids = list(range(1, n_destinations + 1))
        
        synthetic_ratings = []
        random.seed(42)
        
        for _ in range(n_ratings):
            user_id = random.choice(user_ids)
            destination_id = random.choice(destination_ids)
            rating = random.randint(1, 5)
            synthetic_ratings.append({
                'user_id': user_id,
                'destination_id': destination_id,
                'rating': rating
            })
        
        ratings_df = pd.DataFrame(synthetic_ratings)
        print(f"✅ Created synthetic dataset with {len(ratings_df)} ratings")
    
    return ratings_df

async def get_item_features():
    """Get item features dari database atau buat synthetic"""
    try:
        async with get_db() as db:
            query = text("""
            SELECT d.id as destination_id, 
                   COALESCE(c.name, 'Unknown') as category_name,
                   d.name as destination_name
            FROM destinations d
            LEFT JOIN destination_categories dc ON d.id = dc.destination_id
            LEFT JOIN categories c ON c.id = dc.category_id
            """)
            result = await db.execute(query)
            rows = result.fetchall()
            
            item_features = {}
            for row in rows:
                destination_id = row[0]
                category = row[1]
                name = row[2]
                
                item_features[destination_id] = {
                    'category': category,
                    'name': name
                }
            
            print(f"✅ Loaded features for {len(item_features)} destinations from database")
            return item_features
            
    except Exception as e:
        print(f"⚠️ Cannot load item features: {e}. Using synthetic features...")
        
        # Get destinations from ratings
        ratings_df = await load_ratings_data()
        destinations = ratings_df['destination_id'].unique()
        
        categories = ['Wisata Alam', 'Wisata Budaya & Sejarah', 'Wisata Kuliner', 
                     'Wisata Buatan/Rekreasi', 'Wisata Keluarga', 'Wisata Kesehatan & Wellness',
                     'Wisata Petualangan', 'Wisata Religi', 'Wisata Olahraga']
        
        item_features = {}
        for i, destination_id in enumerate(destinations):
            category = categories[i % len(categories)]
            name = f'Destination_{destination_id}'
            item_features[destination_id] = {
                'category': category,
                'name': name
            }
        
        print(f"✅ Created synthetic features for {len(item_features)} destinations")
        return item_features

# Load data
ratings_df = await safe_db_operation(load_ratings_data)
item_features = await safe_db_operation(get_item_features)

# Split data
train_df, test_df = train_test_split(
    ratings_df, 
    test_size=EVALUATION_CONFIG['test_size'], 
    random_state=EVALUATION_CONFIG['random_state'],
    stratify=ratings_df['user_id'].apply(lambda x: min(x % 10, 5))
)

print(f"📊 Data Summary:")
print(f"   Total ratings: {len(ratings_df)}")
print(f"   Train set: {len(train_df)}")
print(f"   Test set: {len(test_df)}")
print(f"   Unique users: {ratings_df['user_id'].nunique()}")
print(f"   Unique destinations: {ratings_df['destination_id'].nunique()}")

# Prepare ground truth
test_truth = test_df.groupby('user_id')['destination_id'].apply(list).to_dict()
eligible_users = [uid for uid in test_truth.keys() if len(test_truth[uid]) >= 2]
eligible_users = eligible_users[:EVALUATION_CONFIG['max_evaluation_users']]

ground_truth_cache = {}
for user_id in eligible_users:
    ground_truth_cache[user_id] = test_truth.get(user_id, [])

print(f"🎯 Evaluation setup:")
print(f"   Eligible users: {len(eligible_users)}")
print(f"   Ground truth items: {sum(len(v) for v in ground_truth_cache.values())}")

In [ ]:
# ===== CELL 6: MODEL TRAINING =====
print("🚀 Training models...")

# Train Collaborative Filtering
cf_model = CollaborativeRecommender()
cf_train_info = await cf_model.train_from_df(train_df)
print(f"✅ Collaborative Filtering trained: {cf_train_info}")

# Train Content-Based
cb_model = ContentBasedRecommender()
await cb_model.train(train_df, item_features)

# Initialize other components
context_component = ContextAwareComponent()
evaluator = ComprehensiveEvaluator(train_df, test_df, item_features)

# Initialize hybrid model
hybrid_model = HybridRecommender(
    cf_model=cf_model,
    cb_model=cb_model,
    context_component=context_component,
    evaluator=evaluator,
    cf_weight=MODEL_WEIGHTS['cf_weight'],
    cb_weight=MODEL_WEIGHTS['cb_weight'],
    context_weight=MODEL_WEIGHTS['context_weight']
)

# Initialize rerankers
mmr_static = MMRReranker(lambda_param=0.5, item_popularity=evaluator.item_popularity)
mmr_adaptive = MMRReranker(lambda_param=0.3, diversity_strength=2.0, item_popularity=evaluator.item_popularity)
mab_optimizer = EnhancedMAB(n_arms=5)

print("✅ All models trained and ready!")

In [ ]:
# ===== CELL 7: STREAMLINED EVALUATION =====
async def run_comprehensive_evaluation():
    """Jalankan evaluasi komprehensif dengan caching"""
    print("🔥 Starting comprehensive evaluation...")
    
    # Cache CF recommendations untuk mempercepat
    print("📦 Caching CF recommendations...")
    cf_cache = {}
    
    for user_id in tqdm(eligible_users, desc="Caching CF"):
        try:
            recs = await cf_model.predict(user_id, EVALUATION_CONFIG['num_recommendations'] * 2)
            cf_cache[user_id] = recs
        except Exception as e:
            print(f"Warning: CF failed for user {user_id}: {e}")
            cf_cache[user_id] = []
    
    # Evaluasi semua model
    print("⚡ Evaluating all models...")
    all_results = []
    
    for user_id in tqdm(eligible_users, desc="Evaluating Users"):
        try:
            context = random.choice(CONTEXTS)
            
            # Get recommendations from all models
            cf_recs = cf_cache.get(user_id, [])
            cb_recs = await cb_model.predict(user_id, EVALUATION_CONFIG['num_recommendations'] * 2)
            hybrid_recs = await hybrid_model.predict(
                user_id, 
                EVALUATION_CONFIG['num_recommendations'] * 2, 
                context, 
                cf_recs,
                item_features
            )
            
            # Apply reranking
            hybrid_mmr_static = await mmr_static.rerank(hybrid_recs, user_id, item_features)
            
            # MAB-adaptive reranking
            selected_arm = mab_optimizer.select_arm(context)
            lambda_param = mab_optimizer.get_lambda_for_mmr(selected_arm)
            mmr_adaptive.lambda_param = lambda_param
            hybrid_mab_mmr = await mmr_adaptive.rerank(hybrid_recs, user_id, item_features)
            
            # Update MAB
            ground_truth = ground_truth_cache.get(user_id, [])
            reward = evaluator.calculate_diversity_reward(
                [r['destination_id'] for r in hybrid_mab_mmr], 
                ground_truth
            )
            mab_optimizer.update(selected_arm, reward, context)
            
            # Store results
            all_results.append({
                'user_id': user_id,
                'recommendations_cf': [r['destination_id'] for r in cf_recs],
                'recommendations_cb': [r['destination_id'] for r in cb_recs],
                'recommendations_hybrid': [r['destination_id'] for r in hybrid_recs],
                'recommendations_hybrid_mmr_static': [r['destination_id'] for r in hybrid_mmr_static],
                'recommendations_hybrid_mab_mmr': [r['destination_id'] for r in hybrid_mab_mmr]
            })
            
        except Exception as e:
            print(f"Error evaluating user {user_id}: {e}")
            continue
    
    evaluation_df = pd.DataFrame(all_results)
    print(f"✅ Evaluation completed! Processed {len(evaluation_df)} users")
    
    return evaluation_df, mab_optimizer

# Jalankan evaluasi
evaluation_df, mab_optimizer = await run_comprehensive_evaluation()

# Simpan hasil
if not evaluation_df.empty:
    evaluation_df.to_pickle('evaluation_results_optimized.pkl')
    print("💾 Results saved to 'evaluation_results_optimized.pkl'")

In [ ]:
# ===== CELL 8: RESULTS ANALYSIS =====
def calculate_comprehensive_metrics(evaluation_df, ground_truth_cache, evaluator):
    """Hitung semua metrics untuk analisis"""
    model_columns = [col for col in evaluation_df.columns if col.startswith('recommendations_')]
    metrics_results = {}
    
    for model_col in model_columns:
        model_name = model_col.replace('recommendations_', '')
        print(f"📊 Calculating metrics for {model_name}...")
        
        precision_scores = []
        recall_scores = []
        ndcg_scores = []
        diversity_scores = []
        novelty_scores = []
        all_recommendations = []
        
        for _, row in evaluation_df.iterrows():
            user_id = row['user_id']
            recs = row[model_col]
            ground_truth = ground_truth_cache.get(user_id, [])
            
            precision_scores.append(evaluator.precision_at_k(recs, ground_truth, 10))
            recall_scores.append(evaluator.recall_at_k(recs, ground_truth, 10))
            ndcg_scores.append(evaluator.ndcg_at_k(recs, ground_truth, 10))
            diversity_scores.append(evaluator.intra_list_diversity(recs, 10))
            novelty_scores.append(evaluator.novelty(recs, 10))
            all_recommendations.append(recs)
        
        coverage = evaluator.coverage_at_k(all_recommendations, 10)
        gini = evaluator.gini_coefficient(all_recommendations)
        
        metrics_results[model_name] = {
            'precision@10': np.mean(precision_scores),
            'recall@10': np.mean(recall_scores),
            'ndcg@10': np.mean(ndcg_scores),
            'diversity': np.mean(diversity_scores),
            'novelty': np.mean(novelty_scores),
            'coverage@10': coverage,
            'gini': gini,
            'precision_scores': precision_scores,
            'diversity_scores': diversity_scores,
            'ndcg_scores': ndcg_scores
        }
    
    return metrics_results

# Calculate metrics
metrics_results = calculate_comprehensive_metrics(evaluation_df, ground_truth_cache, evaluator)

In [ ]:
# ===== CELL 9: VISUALIZATION & STATISTICS =====
def plot_final_results(metrics_results):
    """Plot hasil akhir yang komprehensif"""
    models = list(metrics_results.keys())
    
    # Create subplots
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # Metrics to plot
    metrics_config = [
        ('precision@10', 'Precision@10', axes[0,0]),
        ('recall@10', 'Recall@10', axes[0,1]), 
        ('ndcg@10', 'NDCG@10', axes[0,2]),
        ('diversity', 'Diversity', axes[1,0]),
        ('novelty', 'Novelty', axes[1,1]),
        ('coverage@10', 'Coverage@10', axes[1,2])
    ]
    
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
    
    for i, (metric, title, ax) in enumerate(metrics_config):
        values = [metrics_results[model][metric] for model in models]
        bars = ax.bar(models, values, color=colors[:len(models)], alpha=0.7)
        ax.set_title(title, fontweight='bold')
        ax.tick_params(axis='x', rotation=45)
        
        # Add value labels
        for bar, value in zip(bars, values):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                   f'{value:.3f}', ha='center', va='bottom', fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary table
    print("\\n" + "="*80)
    print("FINAL EVALUATION RESULTS SUMMARY")
    print("="*80)
    
    summary_data = []
    for model in models:
        row = [model]
        for metric in ['precision@10', 'recall@10', 'ndcg@10', 'diversity', 'novelty', 'coverage@10', 'gini']:
            row.append(f"{metrics_results[model][metric]:.4f}")
        summary_data.append(row)
    
    headers = ['Model', 'Precision@10', 'Recall@10', 'NDCG@10', 'Diversity', 'Novelty', 'Coverage@10', 'Gini']
    print(tabulate(summary_data, headers=headers, tablefmt='grid'))
    
    # Statistical significance
    print("\\n" + "="*50)
    print("STATISTICAL SIGNIFICANCE (MAB-MMR vs Others)")
    print("="*50)
    
    target_model = 'hybrid_mab_mmr'
    for metric in ['precision_scores', 'ndcg_scores', 'diversity_scores']:
        print(f"\\n📈 {metric.replace('_scores', '').upper()}:")
        target_scores = metrics_results[target_model][metric]
        
        for baseline in ['cf', 'cb', 'hybrid', 'hybrid_mmr_static']:
            if baseline != target_model:
                baseline_scores = metrics_results[baseline][metric]
                t_stat, p_value = stats.ttest_rel(target_scores, baseline_scores)
                improvement = ((np.mean(target_scores) - np.mean(baseline_scores)) / np.mean(baseline_scores)) * 100
                star = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else ""
                print(f"   {target_model} vs {baseline:20}: p={p_value:.6f} {star} | Improvement: {improvement:+.2f}%")

# Plot results
plot_final_results(metrics_results)

print("🎉 EVALUATION COMPLETED SUCCESSFULLY!")
print("📁 Results saved and visualized!")